In [3]:
import pandas as pd
from modules.lavenshtein import lavenshtein, get_laven
import os
from multiprocessing import Pool
from tqdm import tqdm

In [24]:
users_data = pd.read_csv("data/interim/Users.csv")
with Pool() as p:
    result = p.map(get_laven, users_data.to_dict('records'))
result_df = pd.DataFrame(result)


In [25]:

final_df = result_df

final_df = pd.concat([final_df.reset_index(drop=True),users_data["documentid"].reset_index(drop=True)],axis = 1)
#final_df.to_csv(os.path.join('data/final/', "final.csv"))
display(final_df)
final_df.to_csv(os.path.join('data/final/', "final.csv"))
display(result_df)
for i in result_df['pot_doc']:
    print(len(i))


,pot_doc,pot_name,pot_price,pot_date,pot_address,documentid
0,"[00d0624439175, 00d0876488640, 00d0948483091, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[3, 4, 3, 3, 2, 3, 3, 3, 4, 3]","[7, 8, 8, 7, 7, 8, 6, 8, 8, 7]","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12]",00d0624439175
1,[],[],[],[],[],00d0654270874
2,"[00d0930689679, 00d0734493835, 00d0546591203, ...","[3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 2, 4, 3, 3, 3, 4, 4, 5, 3, 3, 3, 3, 3, 2, ...","[7, 7, 7, 7, 7, 6, 7, 7, 7, 6, 7, 7, 7, 7, 7, ...","[4, 4, 5, 4, 4, 4, 4, 5, 4, 5, 4, 4, 5, 4, 4, ...",00d0403711886
3,[00d0956164753],[4],[5],[6],[21],00d0956164753
4,[00d0667839698],[2],[5],[9],[9],00d0667839698
5,"[00d0602427476, 00d0206117546, 00d0139642641, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[3, 2, 3, 3, 3, 7, 3, 1, 1, 1, 3, 1, 1, 2, 1, ...","[7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[16, 15, 15, 15, 15, 13, 15, 15, 15, 15, 15, 1...",00d0841246769
6,"[00d0164592408, 00d0530724847, 00d0114666542, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2, 2, 2, 2, 2, 1, 3, 2, 2, 3, 2, 2, 2, 2, 3, ...","[7, 7, 8, 7, 8, 7, 10, 8, 7, 10, 7, 8, 8, 7, 1...","[4, 4, 4, 4, 4, 4, 2, 4, 4, 2, 4, 4, 4, 4, 2, ...",00d0414196980
7,"[00d0708763656, 00d0551541808, 00d0921867955, ...","[10, 9, 8, 2, 8, 2, 8, 8, 8, 8, 8, 12, 11, 8, ...","[6, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 4, 2, 3, ...","[7, 7, 8, 6, 8, 8, 6, 8, 7, 7, 5, 7, 10, 8, 9,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, ...",00d0726523245
8,[00d0320587587],[17],[3],[7],[8],00d0320587587
9,"[00d0602427476, 00d0206117546, 00d0139642641, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[2, 2, 1, 3, 3, 6, 3, 2, 2, 2, 1, 2, 2, 2, 2, ...","[8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 8, 7, 8, 8, 7, ...","[15, 14, 14, 14, 14, 12, 14, 14, 14, 14, 14, 1...",00d0333208815


,pot_doc,pot_name,pot_price,pot_date,pot_address
0,"[00d0708763656, 00d0551541808, 00d0921867955, ...","[10, 9, 8, 2, 8, 2, 8, 8, 8, 8, 8, 12, 11, 8, ...","[8, 4, 3, 3, 3, 4, 3, 3, 2, 3, 3, 3, 4, 3, 3, ...","[8, 7, 7, 7, 7, 8, 5, 8, 7, 5, 5, 8, 9, 8, 8, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 4, 3, 3, ..."
1,[00d0599930970],[16],[4],[6],[8]
2,"[00d0930689679, 00d0734493835, 00d0546591203, ...","[3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 2, 4, 3, 3, 3, 4, 4, 5, 3, 3, 3, 3, 3, 2, ...","[7, 7, 7, 7, 7, 6, 7, 7, 7, 6, 7, 7, 7, 7, 7, ...","[6, 8, 8, 8, 8, 8, 8, 8, 6, 8, 8, 8, 8, 8, 8, ..."
3,[00d0956164753],[4],[5],[6],[21]
4,[00d0667839698],[2],[5],[9],[9]
5,"[00d0602427476, 00d0206117546, 00d0139642641, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[3, 2, 3, 3, 3, 7, 3, 1, 1, 1, 3, 1, 1, 2, 1, ...","[7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[16, 15, 15, 15, 15, 13, 15, 15, 15, 15, 15, 1..."
6,"[00d0164592408, 00d0530724847, 00d0114666542, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2, 2, 2, 2, 2, 1, 3, 2, 2, 3, 2, 2, 2, 2, 3, ...","[7, 7, 8, 7, 8, 7, 10, 8, 7, 10, 7, 8, 8, 7, 1...","[4, 4, 4, 4, 4, 4, 2, 4, 4, 2, 4, 4, 4, 4, 2, ..."
7,"[00d0708763656, 00d0551541808, 00d0921867955, ...","[10, 9, 8, 2, 8, 2, 8, 8, 8, 8, 8, 12, 11, 8, ...","[6, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 4, 2, 3, ...","[7, 7, 8, 6, 8, 8, 6, 8, 7, 7, 5, 7, 10, 8, 9,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 5, 5, ..."
8,[00d0320587587],[17],[3],[7],[8]
9,"[00d0602427476, 00d0206117546, 00d0139642641, ...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[2, 2, 1, 3, 3, 6, 3, 2, 2, 2, 1, 2, 2, 2, 2, ...","[8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 8, 7, 8, 8, 7, ...","[15, 14, 14, 14, 14, 12, 14, 14, 14, 14, 14, 1..."


49
1
23
1
1
41
29
49
1
41
48
1
11
49
25
23
5
2
2
1
18
47
12
1
48
49
1
48
18
28
0
18
1
47
23
48
3
1
2
25
25
48
2
48
3
18
41
1
2
41


In [22]:
# new_df = final_df.apply(lambda row: (row['documentid'] in row['doc_id1'])
# | (row['documentid'] in row['doc_id2'])
# | (row['documentid'] in row['doc_id3'])
# | (row['documentid'] in row['doc_id4'])
# | (row['documentid'] in row['doc_id5'])
# , axis=1)
# #display(new_df)
# print(new_df.value_counts()[True])

65
